### The model is softmax

$p(y_i) = \frac{\exp{\beta x_i}}{\sum_i \exp{\beta x_i}}$

That is, we want to relate brain data $x$ of shape [K, F] to stimulus category $y$. The parameters are vector $\beta$ of size F. Sampling can easily give you the uncertainty in $\beta$ given the data and model (sort of using Bayes):

$p(\beta | x) = \frac{p(x | \beta) p(\beta)}{ p(x) }$

Suppose now we know the posterior distribution of $\beta$. The posterior itself $\beta$ is hard to interpret due to its dimensionality, so instead of directly interpreting the parameters we can try to understand how this model works by assessing for every $y$, which $x$ was most likely.

The relation between $y, x, \beta$ is given by softmax, again:

$p(y_i) = \frac{\exp{\beta x_i}}{\sum_i \exp{\beta x_i}}$

so if we want to go from $p(y)$ to $x$ instead of vice versa, we need to rearrange. Take log on both sides

$\ln(p(y_i)) = \beta x_i - \ln{\sum_i \beta x_i }$

$\ln(p(y_i)) + \ln{\sum_i \beta x_i} = \beta x_i$

Since $\ln{\sum_i \beta x_i}$ is constant for all $i$, write $\ln{\sum_i \beta x_i} = c$

$\beta x_i = \ln(p(y_i)) + c$

$x_i = \frac{\ln(p(y_i)) + c}{\beta} = \frac{\ln(p(y_i))}{\beta} + c$

Now I think you can randomly sample (say) 1e4 samples of $\beta$ from your posterior, and for each sample, calculate for all $x_i$ for all $p(y_i) = 0.75$, $i \in \{1...K\}$?

The constant should (hopefully) not be so interesting and can (hopefully) be ignored

In [1]:
import numpy as np
import pymc3 as pm
import seaborn as sns
import theano.tensor as tt
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.linear_model import LogisticRegression

In [823]:
N = 800
P = 2
K = 6

X = np.random.normal(0, 1, size=(N, P))
α = np.random.normal(0, 1, size=K)
β = np.random.normal(0, 1, size=(P, K))
mu = X @ β + α
p_y = softmax(mu, axis=1)

# Make discrete labels
y = np.array([np.random.multinomial(1, p_y[i, :]).argmax() for i in range(N)])
x_ = np.random.normal(0, 1, P)

In [824]:
py_ = softmax(x_ @ β + α)
c = np.log(np.sum(np.exp(x_ @ β + α)))
x_hat = (np.log(py_) - α + c) @ np.linalg.pinv(β)
py_x_hat = softmax(x_hat @ β + α)
np.testing.assert_almost_equal(py_, py_x_hat)

In [826]:
np.set_printoptions(suppress=True)
print(x_hat)
print(x_)

[ 1.29849923 -0.33633367]
[ 1.29849923 -0.33633367]


In [71]:
with pm.Model() as logreg:
    
    α_ = pm.Normal('α_', 0, 1, shape=K)
    β_ = pm.Normal('β_', 0, 1, shape=(P, K))
    mu_ = tt.dot(X, β_) + α_
    p_y_ = tt.nnet.softmax(mu_)
    
    # Do this separately for all classes [0, 1, 2, 3, 4, 5]
    y_ = pm.Categorical('y_', p=p_y_, observed=y)
    trace = pm.sample()

<ipython-input-71-79a4c8661a77>:10: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample()
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [β_, α_]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 120 seconds.


In [87]:
X_ = np.zeros((4000, 50))
for i in range(4000):
    β__ = trace['β_'][i, :, :]
    α__ = trace['α_'][i, :]
    X_[i, :] = (np.log([0.75, 0.05, 0.05, 0.05, 0.05, 0.05]) / (β__ + α__))[:, 0]

array([0.03555049, 0.07532734, 0.05801154, 0.1211306 , 0.37978935,
       0.33019069])

array([ -14.34618063,   15.5755005 , -221.35196824,   10.73879958,
         -6.29797071,  295.51203238,  -46.36097457,  -15.29234092,
        -11.40047414,  -10.59289068,  -17.06979782,   45.2362775 ,
         54.64527373,    9.69887371,   -5.19802335,  -15.52474307,
         11.09071481,   -5.18853588,  -26.34683352,    9.00635544,
        -47.24010665,   -7.17282345,   -6.28379333,   23.47053466,
        -10.11187188,  -12.8351023 ,   12.99048311,    8.28103884,
         49.54233168,   -9.06656821, -171.30894823, -138.14579647,
         14.78052617, -295.735585  ,  138.17967526,  -13.80046704,
         -7.9742874 ,   18.21827718,   -7.30681145,  -12.04135465,
        -52.54368262,   -5.72279662,  -47.54592892,   -6.43465991,
        -13.17473137,  -47.11568827,  -21.45841162,   10.64021153,
         19.9040108 ,  -16.51002614])

In [88]:
np.set_printoptions(suppress=True)
softmax(X_.mean(axis=0) @ β + α)

array([0.00699804, 0.9235571 , 0.00000006, 0.00272476, 0.06514107,
       0.00157897])

In [48]:
X_opt = (np.log([0.75, 0.05, 0.05, 0.05, 0.05, 0.05]) / (β + α))

In [54]:
softmax(X_opt[:, 5] @ β + α).round(3)

array([0., 1., 0., 0., 0., 0.])